#### Задача 1. 

Напишем класс - имитацию класса Token в spacy: у класса должен быть определенный набор атрибутов, создавать новые нельзя. По умолчанию все атрибуты, кроме text, устанавливаются в None, у класса есть методы для установления этих атрибутов (обычные методы экземпляра).

In [5]:
class Token:
    __slots__ = ('text', 'length', 'is_alpha', 'is_punct', 'is_digit')
    def __init__(self, text, length=None, is_alpha=None, is_punct=None, is_digit=None):
        self.text = text
        self.set_length()
        self.set_is_alpha()
        self.set_is_punct()
        self.set_is_digit()

    def set_length(self):
        self.length = len(self.text)

    def set_is_alpha(self):
        self.is_alpha = self.text.isalpha()
    
    def set_is_punct(self):
        self.is_punct = all(char in '.,?!:"' for char in self.text)
    
    def set_is_digit(self):
        self.is_digit = self.text.isdigit()
    
    def __repr__(self):
        return f"Token(text={self.text}, length={self.length}, is_alpha={self.is_alpha}, is_punct={self.is_punct}, is_digit={self.is_digit})\n"

In [7]:
token = Token('aisvnskavjd')
token

Token(text=aisvnskavjd, length=11, is_alpha=True, is_punct=False, is_digit=False)

In [9]:
token.a = 3

AttributeError: 'Token' object has no attribute 'a'

#### Задача 2. 

Добавим нашему токену функциональность: пусть при попытке обратиться к его атрибуту lemma, если таковой не существует, экземпляр класса автоматически вызывает лемматизатор (можно пользоваться pymorphy) и возвращает лемму. Если атрибут lemma уже есть, то достаточно вернуть его содержимое.

In [1]:
import pymorphy3

In [37]:
class Token:
    __slots__ = ('text', 'length', 'is_alpha', 'is_punct', 'is_digit', '_lemma')
    def __init__(self, text, length=None, is_alpha=None, is_punct=None, is_digit=None):
        self.text = text
        self.set_length()
        self.set_is_alpha()
        self.set_is_punct()
        self.set_is_digit()
        self._lemma = None

    def set_length(self):
        self.length = len(self.text)

    def set_is_alpha(self):
        self.is_alpha = self.text.isalpha()
    
    def set_is_punct(self):
        self.is_punct = all(char in '.,?!:"' for char in self.text)
    
    def set_is_digit(self):
        self.is_digit = self.text.isdigit()
    
    def __getattr__(self, attr):
        if attr == 'lemma':
            if self._lemma == None:
                morph = pymorphy3.MorphAnalyzer()
                self._lemma = morph.parse(self.text)[0].normal_form
            return self._lemma
        else:
            raise AttributeError

    # def get_lemma(self):
    #     if self._lemma is None:
    #         morph = pymorphy3.MorphAnalyzer()
    #         self._lemma = morph.parse(self.text)[0].normal_form
    #     return self._lemma
    
    def set_lemma(self, attr):
        self._lemma = attr

    def __repr__(self):
        return f"Token(text={self.text}, length={self.length}, is_alpha={self.is_alpha}, is_punct={self.is_punct}, is_digit={self.is_digit})\n"

In [38]:
token = Token('плачу')
token

Token(text=плачу, length=5, is_alpha=True, is_punct=False, is_digit=False)

In [39]:
token.lemma

'плач'

In [40]:
token.lemma = 'abcd'
token.lemma

AttributeError: 'Token' object has no attribute 'lemma'

#### Задача 3. 

Перепишем задачу 2 с полноценным дескриптором Lemmatize.

In [68]:
class Lemmatize:
    """descriptor"""
    def __get__(self, token, type=None):
        if hasattr(token, '_lemma') and token._lemma is not None: # если есть атрибут _lemma и он не пустой, возвращаем _lemma
            return token._lemma
        else:
            morph = pymorphy3.MorphAnalyzer() # в противном случае — задаём
            token._lemma = morph.parse(token.text)[0].normal_form
            return token._lemma

    def __set__(self, token, value): # нельзя задать атрибут _lemma ручками
        raise AttributeError("Can\'t set!")

In [69]:
class Token:
    __slots__ = ('text', 'length', 'is_alpha', 'is_punct', 'is_digit', '_lemma')
    lemma = Lemmatize() # при создании экзмепляра класса Token у него будет в атрибутах дескриптор

    def __init__(self, text, length=None, is_alpha=None, is_punct=None, is_digit=None):
        self.text = text
        self.set_length()
        self.set_is_alpha()
        self.set_is_punct()
        self.set_is_digit()
        self._lemma = None

    def set_length(self):
        self.length = len(self.text)

    def set_is_alpha(self):
        self.is_alpha = self.text.isalpha()
    
    def set_is_punct(self):
        self.is_punct = all(char in '.,?!:"' for char in self.text)
    
    def set_is_digit(self):
        self.is_digit = self.text.isdigit()

    def __repr__(self):
        return f"Token(text={self.text}, length={self.length}, is_alpha={self.is_alpha}, is_punct={self.is_punct}, is_digit={self.is_digit})\n"

In [70]:
token = Token('плачу')
token

Token(text=плачу, length=5, is_alpha=True, is_punct=False, is_digit=False)

In [71]:
token.lemma

'плач'

In [72]:
token.lemma = 'abcd'

AttributeError: Can't set!

#### Задача 4. 

Напишите класс "пациент", у которого есть устанавливаемые атрибуты "диагноз" и "температура". Атрибут "температура" должен устанавливаться таким образом, чтобы пользователь не мог установить нереалистичное значение типа 15, даже используя синтаксис Patient.temperature = 15.

In [144]:
class Temperature:
    """descriptor"""
    def __get__(self, person, type=None):
        if person is not None:
            if hasattr(person, '_temperature'):
                return person._temperature
        else:
            raise AttributeError
        return self
    
    def __set__(self, person, value):
        if person is None: # запрет на определение метода через класс
            raise AttributeError("Can\'t set temperature at class-level!")        
        if value is None:
            person._temperature = None
        elif 30 <= value <= 42:
            person._temperature = value
        else:
            print('Can\'t set!')

In [145]:
class Patient:
    temperature = Temperature()

    def __init__(self, diagnosis, temperature=None):
        self.diagnosis = diagnosis
        self.__temperature = temperature
    
    def __repr__(self):
        return f"Patient(diagnosis={self.diagnosis}, temperature={self.temperature})"
    
    def __setattr__(self, name, value):
        if name == 'temperature' and isinstance(self, type):
            raise AttributeError("Can\'t set temperature at class-level!")
        super().__setattr__(name, value)

In [146]:
patient = Patient('flu', 37.5)

In [147]:
patient.temperature = 38
patient.temperature

38

In [148]:
patient.temperature = 18

Can't set!


In [149]:
Patient.temperature = 18

#### Задача 5. 

Вернемся к нашему классу Token (первому) и добавим ему метод для подсчета количества созданных токенов. Реализуйте два варианта: через статические методы и методы класса.

In [151]:
class Lemmatize:
    """descriptor"""
    def __get__(self, token, type=None):
        if hasattr(token, '_lemma') and token._lemma is not None: # если есть атрибут _lemma и он не пустой, возвращаем _lemma
            return token._lemma
        else:
            morph = pymorphy3.MorphAnalyzer() # в противном случае — задаём
            token._lemma = morph.parse(token.text)[0].normal_form
            return token._lemma

    def __set__(self, token, value): # нельзя задать атрибут _lemma ручками
        raise AttributeError("Can\'t set!")


class Token:
    __slots__ = ('text', 'length', 'is_alpha', 'is_punct', 'is_digit', '_lemma')
    token_count = 0
    lemma = Lemmatize()

    def __init__(self, text, length=None, is_alpha=None, is_punct=None, is_digit=None):
        self.text = text
        self.set_length()
        self.set_is_alpha()
        self.set_is_punct()
        self.set_is_digit()
        self._lemma = None
        Token.increment_token_count() # счетчик увеличивается каждый раз, когда создается экземпляр
    
    @classmethod
    def increment_token_count(cls): # метод класса для увеличения счетчика токенов
        cls.token_count += 1

    @classmethod
    def get_token_count(cls): # метод класса для получения текущего кол-ва токенов
        return cls.token_count

    def set_length(self):
        self.length = len(self.text)

    def set_is_alpha(self):
        self.is_alpha = self.text.isalpha()
    
    def set_is_punct(self):
        self.is_punct = all(char in '.,?!:"' for char in self.text)
    
    def set_is_digit(self):
        self.is_digit = self.text.isdigit()

    def __repr__(self):
        return f"Token(text={self.text}, length={self.length}, is_alpha={self.is_alpha}, is_punct={self.is_punct}, is_digit={self.is_digit})\n"

In [152]:
token1 = Token('плачу')
token2 = Token('лечу')
token3 = Token('стали')

In [153]:
Token.get_token_count()

3

In [154]:
class Lemmatize:
    """descriptor"""
    def __get__(self, token, type=None):
        if hasattr(token, '_lemma') and token._lemma is not None: # если есть атрибут _lemma и он не пустой, возвращаем _lemma
            return token._lemma
        else:
            morph = pymorphy3.MorphAnalyzer() # в противном случае — задаём
            token._lemma = morph.parse(token.text)[0].normal_form
            return token._lemma

    def __set__(self, token, value): # нельзя задать атрибут _lemma ручками
        raise AttributeError("Can\'t set!")


class Token:
    __slots__ = ('text', 'length', 'is_alpha', 'is_punct', 'is_digit', '_lemma')
    token_count = 0
    lemma = Lemmatize()

    def __init__(self, text, length=None, is_alpha=None, is_punct=None, is_digit=None):
        self.text = text
        self.set_length()
        self.set_is_alpha()
        self.set_is_punct()
        self.set_is_digit()
        self._lemma = None
        Token.increment_token_count() # счетчик увеличивается каждый раз, когда создается экземпляр
    
    @staticmethod
    def increment_token_count(): # метод класса для увеличения счетчика токенов
        Token.token_count += 1

    @staticmethod
    def get_token_count(): # метод класса для получения текущего кол-ва токенов
        return Token.token_count

    def set_length(self):
        self.length = len(self.text)

    def set_is_alpha(self):
        self.is_alpha = self.text.isalpha()
    
    def set_is_punct(self):
        self.is_punct = all(char in '.,?!:"' for char in self.text)
    
    def set_is_digit(self):
        self.is_digit = self.text.isdigit()

    def __repr__(self):
        return f"Token(text={self.text}, length={self.length}, is_alpha={self.is_alpha}, is_punct={self.is_punct}, is_digit={self.is_digit})\n"

In [155]:
token1 = Token('плачу')
token2 = Token('лечу')
token3 = Token('стали')

In [156]:
Token.get_token_count()

3